In [1]:
import pandas as pd
import numpy as np

In [2]:
infores = pd.read_csv('2022-11-07.csv')
infores = infores.drop(columns=['brief_description', 'country', 'province', 'district', 'town_city', 'arms_munition', 'violence_level', 'geolocation'])
print(f"entries: {str(len(infores))}")
infores.dropna(
  axis=0,
  how='any',
  thresh=None,
  subset=None,
  inplace=True
)
print(f"valid entries: {str(len(infores))}")

event_categories = infores['event_group'].unique()
event_categories = np.array([(x, i + 1) for i, x in enumerate(event_categories)])
event_categories

entries: 6431
valid entries: 6382


array([['Russian Military Presence January 2022', '1'],
       ['Russian Allies Presence/Losses', '2'],
       ['Russian Military Presence February 2022', '3'],
       ['Other', '4'],
       ['Civilian Infrastructure Damage March 2022', '5'],
       ['Bombing or Explosion Feb - June', '6'],
       ['Civilian Infrastructure Damage February 2022', '7'],
       ['Civilian Casualty', '8'],
       ['Ukrainian Military Losses', '9'],
       ['Russian Firing Positions', '10'],
       ['Ground Battle', '11'],
       ['Military Infrastructure Damage', '12'],
       ['Russian Military Losses Feb - June', '13'],
       ['Munitions', '14'],
       ['Russian Military Presence March 2022', '15'],
       ['Detentions or Arrests', '16'],
       ['Mass grave or Burial', '17'],
       ['Civilian Infrastructure Damage April 2022', '18'],
       ['Russian Military Presence April 2022', '19'],
       ['Civilian Infrastructure Damage May 2022', '20'],
       ['Russian Military Presence May 2022', '21'],
   

In [ ]:
from DBEngine import db_engine

def empty(cursor):
  query = "select (select count(1) from info_res) = 0"
  cursor.execute(query)
  return cursor.fetchone()

if(db_engine.execute(empty)[0]):
  def insert(cursor):
    args_str = ','.join([cursor.mogrify("(%s,%s)", (x[1], x[0])).decode('utf-8') for x in event_categories])
    query = "insert into info_res_event_categories values " + args_str
    cursor.execute(query)

  db_engine.execute(insert)
  db_engine.commit()

  def insert(cursor):
    categories = dict(event_categories)
    infores['event_group'] = [int(categories[x]) for x in infores['event_group']]
    query = "insert into info_res (entry,date,event_group,title,link,point) values (%(entry)s,%(date)s,%(event_group)s,%(title)s,%(link)s,ST_Point(%(lon)s,%(lat)s,4326))"
    cursor.executemany(query, infores.to_dict('records'))

  db_engine.execute(insert)
  db_engine.commit()